<a href="https://colab.research.google.com/github/vitakurakina/ailabs/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22lab1_regression_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа № 1 Задача регрессии

## Задание
Решите пожалуйста следующие задачи.

1. Примените следующие типы регрессий для датасета представленного по
[ссылке](https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link).
- Линейная регрессия
- LASSO регрессия
- Ridge регрессия
- ElasticNet регрессия
- Polynom регрессия (для степени полинома 2)
- Polynom регрессия (для степени полинома 5)
- Polynom регрессия (для степени полинома 10)

Постройте модель для атрибутов `X = {SkinThickness, BMI}` и `Y = {Insulin}`.  
**Укажите `score` (r2_score) моделей. Найдите метрики MAE, MSE.**


## Загрузка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
! gdown --fuzzy 'https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link'

Downloading...
From: https://drive.google.com/uc?id=1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec
To: /content/diabetes.csv
100% 23.1k/23.1k [00:00<00:00, 51.9MB/s]


In [ ]:
df = pd.read_csv('/content/diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


##  О датасете
[Pima Indians Diabetes Database](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data)

Этот набор данных взят из Национального института диабета, заболеваний органов пищеварения и почек (NIDDK). Целью набора данных является диагностическое предсказание наличия у пациента диабета на основе определенных диагностических измерений, включенных в набор данных. На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года, родом из индейцев Пима.

Наборы данных состоят из нескольких переменных медицинских предикторов и одной целевой переменной `Outcome`. Переменные-предикторы включают количество беременностей, которые были у пациентки, её индекс массы тела `BMI`, уровень инсулина, возраст и так далее. Всего в датасете 9 переменных:
- `Pregnancies` - количество беременностей;
- `Glucose` - концентрация глюкозы в плазме через 2 часа при пероральном тесте на толерантность к глюкозе;
- `BloodPressure` - диастолическое артериальное давление (мм рт. ст.);
- `SkinThickness` - толщина кожной складки трицепса (мм);
- `Insulin` - 2-часовой сывороточный инсулин (мкЕд/мл);
- `BMI` - Индекс массы тела (вес в кг/(рост в м)^2);
- `DiabetesPedigreeFunction` - функция оценки вероятности диабета на основе семейного анамнеза;
- `Age` - возраст (лет);
- `Outcome` - целевая переменная (0 или 1) 268 из 768 — 1 (больны), остальные — 0 (здоровы).



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

## Линейная регрессия

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Prepare data
X = df[['SkinThickness', 'BMI']]
Y = df['Insulin']

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create and train the model
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)

# Make predictions
Y_pred = linear_model.predict(X_test)

# Evaluate the model
r2 = r2_score(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)

print(f'Linear Regression:')
print(f'  R2 Score: {r2:.4f}')
print(f'  MAE: {mae:.4f}')
print(f'  MSE: {mse:.4f}')

Linear Regression:
  R2 Score: 0.1812
  MAE: 67.2310
  MSE: 10083.7350


## LASSO регрессия

In [ ]:
from sklearn.linear_model import Lasso

# Create and train the model
lasso_model = Lasso(alpha=1.0) # You can adjust alpha
lasso_model.fit(X_train, Y_train)

# Make predictions
Y_pred_lasso = lasso_model.predict(X_test)

# Evaluate the model
r2_lasso = r2_score(Y_test, Y_pred_lasso)
mae_lasso = mean_absolute_error(Y_test, Y_pred_lasso)
mse_lasso = mean_squared_error(Y_test, Y_pred_lasso)

print(f'LASSO Regression:')
print(f'  R2 Score: {r2_lasso:.4f}')
print(f'  MAE: {mae_lasso:.4f}')
print(f'  MSE: {mse_lasso:.4f}')

LASSO Regression:
  R2 Score: 0.1813
  MAE: 67.2542
  MSE: 10082.0923


## Ridge регрессия

In [ ]:
from sklearn.linear_model import Ridge

# Create and train the model
ridge_model = Ridge(alpha=1.0) # You can adjust alpha
ridge_model.fit(X_train, Y_train)

# Make predictions
Y_pred_ridge = ridge_model.predict(X_test)

# Evaluate the model
r2_ridge = r2_score(Y_test, Y_pred_ridge)
mae_ridge = mean_absolute_error(Y_test, Y_pred_ridge)
mse_ridge = mean_squared_error(Y_test, Y_pred_ridge)

print(f'Ridge Regression:')
print(f'  R2 Score: {r2_ridge:.4f}')
print(f'  MAE: {mae_ridge:.4f}')
print(f'  MSE: {mse_ridge:.4f}')

Ridge Regression:
  R2 Score: 0.1812
  MAE: 67.2311
  MSE: 10083.7272


## ElasticNet регрессия

In [ ]:
from sklearn.linear_model import ElasticNet

# Create and train the model
elasticnet_model = ElasticNet(alpha=1.0, l1_ratio=0.5) # You can adjust alpha and l1_ratio
elasticnet_model.fit(X_train, Y_train)

# Make predictions
Y_pred_elasticnet = elasticnet_model.predict(X_test)

# Evaluate the model
r2_elasticnet = r2_score(Y_test, Y_pred_elasticnet)
mae_elasticnet = mean_absolute_error(Y_test, Y_pred_elasticnet)
mse_elasticnet = mean_squared_error(Y_test, Y_pred_elasticnet)

print(f'ElasticNet Regression:')
print(f'  R2 Score: {r2_elasticnet:.4f}')
print(f'  MAE: {mae_elasticnet:.4f}')
print(f'  MSE: {mse_elasticnet:.4f}')

ElasticNet Regression:
  R2 Score: 0.1814
  MAE: 67.2802
  MSE: 10080.5488


## Polynom регрессия (для степени полинома 2)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Create polynomial features
poly_features_2 = PolynomialFeatures(degree=2)
X_train_poly_2 = poly_features_2.fit_transform(X_train)
X_test_poly_2 = poly_features_2.transform(X_test)

# Create and train the model
poly_model_2 = LinearRegression()
poly_model_2.fit(X_train_poly_2, Y_train)

# Make predictions
Y_pred_poly_2 = poly_model_2.predict(X_test_poly_2)

# Evaluate the model
r2_poly_2 = r2_score(Y_test, Y_pred_poly_2)
mae_poly_2 = mean_absolute_error(Y_test, Y_pred_poly_2)
mse_poly_2 = mean_squared_error(Y_test, Y_pred_poly_2)

print(f'Polynomial Regression (Degree 2):')
print(f'  R2 Score: {r2_poly_2:.4f}')
print(f'  MAE: {mae_poly_2:.4f}')
print(f'  MSE: {mse_poly_2:.4f}')

Polynomial Regression (Degree 2):
  R2 Score: 0.1680
  MAE: 66.2456
  MSE: 10245.4591


## Polynom регрессия (для степени полинома 5)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Create polynomial features
poly_features_5 = PolynomialFeatures(degree=5)
X_train_poly_5 = poly_features_5.fit_transform(X_train)
X_test_poly_5 = poly_features_5.transform(X_test)

# Create and train the model
poly_model_5 = LinearRegression()
poly_model_5.fit(X_train_poly_5, Y_train)

# Make predictions
Y_pred_poly_5 = poly_model_5.predict(X_test_poly_5)

# Evaluate the model
r2_poly_5 = r2_score(Y_test, Y_pred_poly_5)
mae_poly_5 = mean_absolute_error(Y_test, Y_pred_poly_5)
mse_poly_5 = mean_squared_error(Y_test, Y_pred_poly_5)

print(f'Polynomial Regression (Degree 5):')
print(f'  R2 Score: {r2_poly_5:.4f}')
print(f'  MAE: {mae_poly_5:.4f}')
print(f'  MSE: {mse_poly_5:.4f}')

Polynomial Regression (Degree 5):
  R2 Score: -102.1129
  MAE: 153.2557
  MSE: 1269818.1114


## Polynom регрессия (для степени полинома 10)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Create polynomial features
poly_features_10 = PolynomialFeatures(degree=10)
X_train_poly_10 = poly_features_10.fit_transform(X_train)
X_test_poly_10 = poly_features_10.transform(X_test)

# Create and train the model
poly_model_10 = LinearRegression()
poly_model_10.fit(X_train_poly_10, Y_train)

# Make predictions
Y_pred_poly_10 = poly_model_10.predict(X_test_poly_10)

# Evaluate the model
r2_poly_10 = r2_score(Y_test, Y_pred_poly_10)
mae_poly_10 = mean_absolute_error(Y_test, Y_pred_poly_10)
mse_poly_10 = mean_squared_error(Y_test, Y_pred_poly_10)

print(f'Polynomial Regression (Degree 10):')
print(f'  R2 Score: {r2_poly_10:.4f}')
print(f'  MAE: {mae_poly_10:.4f}')
print(f'  MSE: {mse_poly_10:.4f}')

Polynomial Regression (Degree 10):
  R2 Score: -30000512581.1079
  MAE: 1548976.1908
  MSE: 369451263871929.6250


Based on the calculated metrics:

*   **Linear Regression, LASSO, Ridge, and ElasticNet** all have similar low R2 scores (around 0.18) and relatively high MAE and MSE values. This suggests that a simple linear relationship between `SkinThickness`, `BMI`, and `Insulin` is not strong, and these models do not capture much of the variance in the `Insulin` data. The regularization in LASSO, Ridge, and ElasticNet did not significantly improve the performance in this case, likely because the initial linear model was already underfitting.

*   **Polynomial Regression (Degree 2)** has a slightly lower R2 score (around 0.168) and similar MAE and MSE compared to the linear models. This indicates that introducing polynomial features of degree 2 did not significantly improve the model's ability to fit the data.

*   **Polynomial Regression (Degree 5 and 10)** show highly negative R2 scores and very large MAE and MSE values. This is a clear indication of severe overfitting. The models are too complex and are fitting the noise in the training data rather than the underlying relationship, leading to extremely poor performance on the test data.

**Conclusion:**

For this specific task of predicting `Insulin` based on `SkinThickness` and `BMI`, none of the applied regression models performed well. The low R2 scores across the linear and degree 2 polynomial models suggest that `SkinThickness` and `BMI` alone are not strong predictors of `Insulin` in this dataset, or that the relationship is highly non-linear and not well captured by these models. The higher degree polynomial models (5 and 10) clearly overfit the data, demonstrating that increasing model complexity without sufficient underlying patterns or data can lead to worse results.

It would be beneficial to explore other features in the dataset, consider different regression algorithms, or investigate potential data issues (e.g., outliers, missing values) that might be affecting model performance.

## Вывод

Сделайте вывод о применимости разных видов регрессии для решения этой задачи.

Основывваясь на вычислениях:

*   Линейная регрессия, Ridge и ElasticNet имеют одинакого низкий R2 баллы (около 0.18) и схожие высокие MAE и MSE значения. Это предполагает что обычные линейные отношения между SkinThickness, BMI и Insulin не такие сильные, и эти модели не могут запечатлеть много вариаций Insulin данных. Регуляризация в LASSO, Ridge и ElasticNet не улучшила производительность в этом случае, вероятно потомк что линейная модель была переобучена.

*   Полиноминальная регрессия (Степень 2) имеет довольно маленький R2 значение (около 0.168) и одинаковые MAE и MSE значения в отличие от линейной можели. Это означает что введение полиноминальных характеристик не улучшили возможность модели улучшить данные.

* Полиноминальная регрессия (степени 5 и 10) имеют большие отрицательные R2 значения и очень большие значения MAE и MSE. Это явно показывает переобучение. Модели слишком сложны и подгоняют шум в обучающих данных, а не лежащую в их основе взаимосвязь, что приводит к крайне низкой производительности на тестовых данных.

Вывод:

Для специфической задачи прогноза Инсулина, базирующейся на SkinThickness и BMI, ни одна из приложенных регрессионных моделей не работает хорошо. Низкое значение R2 в линейной и полиноминальной регрессии со степенью 2 предполагает что SkinThickness и BMI не являются хорошими предсказателями Инсулина в этом датасете, или что взаимосвязь не носит линейных характер или плохо отражается моделями. В регрессионных моделях с более высокими степенями (5 и 10) явно переобучаются данные, демонстрируя что увеличение сложности модели без достаточного количества паттернов или данных могут привести к худшим результатам.

Было бы полезно изучить другие характеристики набора данных, рассмотреть различные алгоритмы регрессии или исследовать потенциальные проблемы с данными (например, выбросы, пропущенные значения), которые могут влиять на производительность модели.
